In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option("display.max_rows", 150)

/Users/hiromi/.pyenv/versions/3.8.10/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Scraping

## 1. Amazon

In [2]:
url1 = 'https://lda.senate.gov/filings/public/filing/search/?registrant=amazon&registrant_country=&registrant_ppb_country=&client=&client_state=&client_country=&client_ppb_country=&lobbyist=&lobbyist_covered_position=&lobbyist_conviction_disclosure=&lobbyist_conviction_date_range_from=&lobbyist_conviction_date_range_to=&report_period=&report_year=&report_dt_posted_from=&report_dt_posted_to=&report_amount_reported_min=&report_amount_reported_max=&report_issue_area_description=&affiliated_organization=&affiliated_organization_country=&foreign_entity=&foreign_entity_country=&foreign_entity_ppb_country=&foreign_entity_ownership_percentage_min=&foreign_entity_ownership_percentage_max=&search=search#js_searchFormTitle'

In [3]:
#use easy scraping way io.html.read_html()
df1 = pd.io.html.read_html(url1)
df_amazon = df1[0]
df_amazon = df_amazon.sort_values(by = 'Filing Year', ascending=False)
df_amazon['AmountReported'] = df_amazon['AmountReported'].str.replace('$', '', regex=True).str.replace(',','').astype(float)
df_amazon['season'] = df_amazon['Filing Year'].astype(str)  + ' ' +  df_amazon['Report Type'] 

In [4]:
# get the link info 
response = requests.get(url1)
doc = BeautifulSoup(response.text, 'html.parser')
lobbylists = doc.select('.tr--clickable')
amazon_lobby_list = []
for lobbylist in lobbylists:
    dict = {}
    dict['Posted'] = lobbylist.select_one('.col-w--sm').text.strip()
    dict['link'] = 'https://lda.senate.gov/' + lobbylist.select_one('.js_clickableRowTarget')['href']    
    amazon_lobby_list.append(dict)

In [5]:
#DataFrame of PDF link to Amazon's lobby
df_amazon_link = pd.DataFrame(amazon_lobby_list)

In [6]:
print(df_amazon.shape)
print(df_amazon_link.shape)

(79, 7)
(79, 2)


In [7]:
#merge 2 DataFrame
df_amazon = df_amazon.merge(df_amazon_link, how='left', on='Posted')

In [8]:
#Pick important columns
df_Amazon = df_amazon.rename(columns={'AmountReported' : 'amazon_lobby', 'link': 'amazon_link'})
df_Amazon = df_Amazon[['Registrant Name', 'season','amazon_lobby','amazon_link']]

In [9]:
df_Amazon.head(2)

,Registrant Name,season,amazon_lobby,amazon_link
0,AMAZON.COM SERVICES LLC,2021 2nd Quarter - Report,4860000.0,https://lda.senate.gov//filings/public/filing/...
1,AMAZON.COM SERVICES LLC,2021 1st Quarter - Report,4800000.0,https://lda.senate.gov//filings/public/filing/...


## 2. Facebook

In [10]:
url2 = 'https://lda.senate.gov/filings/public/filing/search/?registrant=facebook&registrant_country=&registrant_ppb_country=&client=&client_state=&client_country=&client_ppb_country=&lobbyist=&lobbyist_covered_position=&lobbyist_conviction_disclosure=&lobbyist_conviction_date_range_from=&lobbyist_conviction_date_range_to=&report_period=&report_year=&report_dt_posted_from=&report_dt_posted_to=&report_amount_reported_min=&report_amount_reported_max=&report_issue_area_description=&affiliated_organization=&affiliated_organization_country=&foreign_entity=&foreign_entity_country=&foreign_entity_ppb_country=&foreign_entity_ownership_percentage_min=&foreign_entity_ownership_percentage_max=&search=search#js_searchFormTitle'

In [11]:
#use easy scraping way io.html.read_html()
df2 = pd.io.html.read_html(url2)
df_facebook = df2[0]
df_facebook = df_facebook.sort_values(by = 'Filing Year', ascending=False)
df_facebook['AmountReported'] = df_facebook['AmountReported'].str.replace('$', '', regex=True).str.replace(',','').astype(float)
df_facebook['season'] = df_facebook['Filing Year'].astype(str)  + ' ' +  df_facebook['Report Type'] 

In [12]:
# get the link info 
response = requests.get(url2)
doc = BeautifulSoup(response.text, 'html.parser')
lobbylists = doc.select('.tr--clickable')
facebook_lobby_list = []
for lobbylist in lobbylists:
    dict = {}
    dict['Posted'] = lobbylist.select_one('.col-w--sm').text.strip()
    dict['link'] = 'https://lda.senate.gov/' + lobbylist.select_one('.js_clickableRowTarget')['href']    
    facebook_lobby_list.append(dict)

In [13]:
#DataFrame of PDF link to FB's lobby
df_facebook_link = pd.DataFrame(facebook_lobby_list)

In [14]:
print(df_facebook.shape)
print(df_facebook_link.shape)

(53, 7)
(53, 2)


In [15]:
#merge 2 DataFrame
df_facebook = df_facebook.merge(df_facebook_link, how='left', on='Posted')

In [16]:
#Pick important columns
df_Facebook = df_facebook.rename(columns={'AmountReported' : 'facebook_lobby', 'link': 'facebook_link'})
df_Facebook = df_Facebook[['Registrant Name', 'season','facebook_lobby','facebook_link']]

In [17]:
df_Facebook.head()

,Registrant Name,season,facebook_lobby,facebook_link
0,"FACEBOOK, INC. AND VARIOUS SUBSIDIARIES",2021 2nd Quarter - Report,4770000.0,https://lda.senate.gov//filings/public/filing/...
1,"FACEBOOK, INC. AND VARIOUS SUBSIDIARIES",2021 1st Quarter - Report,4790000.0,https://lda.senate.gov//filings/public/filing/...
2,"FACEBOOK, INC. AND VARIOUS SUBSIDIARIES",2020 1st Quarter - Report,5260000.0,https://lda.senate.gov//filings/public/filing/...
3,"FACEBOOK, INC. AND VARIOUS SUBSIDIARIES",2020 4th Quarter - Report,4690000.0,https://lda.senate.gov//filings/public/filing/...
4,"FACEBOOK, INC. AND VARIOUS SUBSIDIARIES",2020 3rd Quarter - Amendment,4900000.0,https://lda.senate.gov//filings/public/filing/...


## 3. Google

In [18]:
url3 = 'https://lda.senate.gov/filings/public/filing/search/?registrant=google&registrant_country=&registrant_ppb_country=&client=&client_state=&client_country=&client_ppb_country=&lobbyist=&lobbyist_covered_position=&lobbyist_conviction_disclosure=&lobbyist_conviction_date_range_from=&lobbyist_conviction_date_range_to=&report_period=&report_year=&report_dt_posted_from=&report_dt_posted_to=&report_amount_reported_min=&report_amount_reported_max=&report_issue_area_description=&affiliated_organization=&affiliated_organization_country=&foreign_entity=&foreign_entity_country=&foreign_entity_ppb_country=&foreign_entity_ownership_percentage_min=&foreign_entity_ownership_percentage_max=&search=search#js_searchFormTitle'

In [19]:
#use easy scraping way io.html.read_html()
df3 = pd.io.html.read_html(url3)
df_google = df3[0]
df_google = df_google.sort_values(by = 'Filing Year', ascending=False)
df_google['AmountReported'] = df_google['AmountReported'].str.replace('$', '', regex=True).str.replace(',','').astype(float)
df_google['season'] = df_google['Filing Year'].astype(str)  + ' ' +  df_google['Report Type'] 

In [20]:
# get the link info 
response = requests.get(url3)
doc = BeautifulSoup(response.text, 'html.parser')
lobbylists = doc.select('.tr--clickable')
google_lobby_list = []
for lobbylist in lobbylists:
    dict = {}
    dict['Posted'] = lobbylist.select_one('.col-w--sm').text.strip()
    dict['link'] = 'https://lda.senate.gov/' + lobbylist.select_one('.js_clickableRowTarget')['href']    
    google_lobby_list.append(dict)

In [21]:
#DataFrame of PDF link to google's lobby
df_google_link = pd.DataFrame(google_lobby_list)

In [22]:
print(df_google.shape)
print(df_google_link.shape)

(63, 7)
(63, 2)


In [23]:
#merge 2 DataFrame
df_google = df_google.merge(df_google_link, how='left', on='Posted')

In [24]:
#Pick important columns
df_Google = df_google.rename(columns={'AmountReported' : 'google_lobby', 'link': 'google_link'})
df_Google = df_Google[['Registrant Name', 'season','google_lobby','google_link']]

In [25]:
df_Google.head()

,Registrant Name,season,google_lobby,google_link
0,GOOGLE CLIENT SERVICES LLC,2021 2nd Quarter - Report,2090000.0,https://lda.senate.gov//filings/public/filing/...
1,GOOGLE CLIENT SERVICES LLC,2021 1st Quarter - Report,2690000.0,https://lda.senate.gov//filings/public/filing/...
2,GOOGLE CLIENT SERVICES LLC,2020 4th Quarter - Report,2110000.0,https://lda.senate.gov//filings/public/filing/...
3,GOOGLE CLIENT SERVICES LLC,2020 3rd Quarter - Report,1930000.0,https://lda.senate.gov//filings/public/filing/...
4,GOOGLE CLIENT SERVICES LLC,2020 2nd Quarter - Report,1690000.0,https://lda.senate.gov//filings/public/filing/...


### 4. Apple

In [26]:
url4 = 'https://lda.senate.gov/filings/public/filing/search/?registrant=apple+inc&registrant_country=&registrant_ppb_country=&client=&client_state=&client_country=&client_ppb_country=&lobbyist=&lobbyist_covered_position=&lobbyist_conviction_disclosure=&lobbyist_conviction_date_range_from=&lobbyist_conviction_date_range_to=&report_period=&report_year=&report_dt_posted_from=&report_dt_posted_to=&report_amount_reported_min=&report_amount_reported_max=&report_issue_area_description=&affiliated_organization=&affiliated_organization_country=&foreign_entity=&foreign_entity_country=&foreign_entity_ppb_country=&foreign_entity_ownership_percentage_min=&foreign_entity_ownership_percentage_max=&search=search#js_searchFormTitle'

In [27]:
#use easy scraping way io.html.read_html()
df4 = pd.io.html.read_html(url4)
df_apple = df4[0]
df_apple = df_apple.sort_values(by = 'Filing Year', ascending=False)
df_apple['AmountReported'] = df_apple['AmountReported'].str.replace('$', '', regex=True).str.replace(',','').astype(float)
df_apple['season'] = df_apple['Filing Year'].astype(str)  + ' ' +  df_apple['Report Type'] 

In [28]:
# get the link info 
response = requests.get(url4)
doc = BeautifulSoup(response.text, 'html.parser')
lobbylists = doc.select('.tr--clickable')
apple_lobby_list = []
for lobbylist in lobbylists:
    dict = {}
    dict['Posted'] = lobbylist.select_one('.col-w--sm').text.strip()
    dict['link'] = 'https://lda.senate.gov/' + lobbylist.select_one('.js_clickableRowTarget')['href']    
    apple_lobby_list.append(dict)

In [29]:
#DataFrame of PDF link to google's lobby
df_apple_link = pd.DataFrame(apple_lobby_list)

In [30]:
print(df_apple.shape)
print(df_apple_link.shape)

(89, 7)
(89, 2)


In [31]:
#merge 2 DataFrame
df_apple = df_apple.merge(df_apple_link, how='left', on='Posted')

In [32]:
#Pick important columns
df_Apple = df_apple.rename(columns={'AmountReported' : 'apple_lobby', 'link': 'apple_link'})
df_Apple = df_Apple[['Registrant Name', 'season','apple_lobby','apple_link']]

In [33]:
df_Apple.head()

,Registrant Name,season,apple_lobby,apple_link
0,APPLE INC.,2021 1st Quarter - Report,1460000.0,https://lda.senate.gov//filings/public/filing/...
1,APPLE INC.,2021 2nd Quarter - Report,1640000.0,https://lda.senate.gov//filings/public/filing/...
2,APPLE INC.,2020 3rd Quarter - Amendment,1560000.0,https://lda.senate.gov//filings/public/filing/...
3,APPLE INC.,2020 3rd Quarter - Report,1560000.0,https://lda.senate.gov//filings/public/filing/...
4,APPLE INC.,2020 4th Quarter - Report,1450000.0,https://lda.senate.gov//filings/public/filing/...


## 5. Microsoft

In [34]:
url5 = 'https://lda.senate.gov/filings/public/filing/search/?registrant=microsoft&registrant_country=&registrant_ppb_country=&client=&client_state=&client_country=&client_ppb_country=&lobbyist=&lobbyist_covered_position=&lobbyist_conviction_disclosure=&lobbyist_conviction_date_range_from=&lobbyist_conviction_date_range_to=&report_period=&report_year=&report_dt_posted_from=&report_dt_posted_to=&report_amount_reported_min=&report_amount_reported_max=&report_issue_area_description=&affiliated_organization=&affiliated_organization_country=&foreign_entity=&foreign_entity_country=&foreign_entity_ppb_country=&foreign_entity_ownership_percentage_min=&foreign_entity_ownership_percentage_max=&search=search#js_searchFormTitle'

In [35]:
#use easy scraping way io.html.read_html()
df5 = pd.io.html.read_html(url5)
df_microsoft = df5[0]
df_microsoft = df_microsoft.sort_values(by = 'Filing Year', ascending=False)
df_microsoft['AmountReported'] = df_microsoft['AmountReported'].str.replace('$', '', regex=True).str.replace(',','').astype(float)
df_microsoft['season'] = df_microsoft['Filing Year'].astype(str)  + ' ' +  df_microsoft['Report Type'] 

In [36]:
# get the link info 
response = requests.get(url5)
doc = BeautifulSoup(response.text, 'html.parser')
lobbylists = doc.select('.tr--clickable')
msft_lobby_list = []
for lobbylist in lobbylists:
    dict = {}
    dict['Posted'] = lobbylist.select_one('.col-w--sm').text.strip()
    dict['link'] = 'https://lda.senate.gov/' + lobbylist.select_one('.js_clickableRowTarget')['href']    
    msft_lobby_list.append(dict)

In [37]:
#DataFrame of PDF link to google's lobby
df_microsoft_link = pd.DataFrame(msft_lobby_list)

In [38]:
print(df_microsoft.shape)
print(df_microsoft_link.shape)

(76, 7)
(76, 2)


In [39]:
#merge 2 DataFrame
df_microsoft = df_microsoft.merge(df_microsoft_link, how='left', on='Posted')

In [40]:
#Pick important columns
df_Microsoft = df_microsoft.rename(columns={'AmountReported' : 'microsoft_lobby', 'link': 'microsoft_link'})
df_Microsoft = df_Microsoft[['Registrant Name', 'season','microsoft_lobby','microsoft_link']]

In [41]:
df_Microsoft.head()

,Registrant Name,season,microsoft_lobby,microsoft_link
0,MICROSOFT CORPORATION,2021 2nd Quarter - Report,2470000.0,https://lda.senate.gov//filings/public/filing/...
1,MICROSOFT CORPORATION,2021 1st Quarter - Report,2590000.0,https://lda.senate.gov//filings/public/filing/...
2,MICROSOFT CORPORATION,2020 4th Quarter - Report,2190000.0,https://lda.senate.gov//filings/public/filing/...
3,MICROSOFT CORPORATION,2020 3rd Quarter - Report,1880000.0,https://lda.senate.gov//filings/public/filing/...
4,MICROSOFT CORPORATION,2020 2nd Quarter - Report,2910000.0,https://lda.senate.gov//filings/public/filing/...


## 6. Bigtech (combined table)

In [42]:
df_bigtech = df_Amazon.merge(df_Facebook, on = ['season'], how = 'outer')\
    .merge(df_Google, on = ['season'], how = 'outer')\
    .merge(df_Apple, on = ['season'], how = 'outer')\
    .merge(df_Microsoft, on = ['season'], how = 'outer')

In [43]:
df_bigtech.shape

(148, 16)

In [60]:
# df_bigtech.columns

In [44]:
df_Bigtech = df_bigtech[['season', 'amazon_lobby','facebook_lobby','google_lobby','apple_lobby','microsoft_lobby', \
                        'amazon_link','facebook_link','google_link','apple_link','microsoft_link']]

In [45]:
df_Bigtech = df_Bigtech.sort_values(by='season', ascending=False).fillna(0)

In [48]:
df_Bigtech['year'] = df_Bigtech['season'].str[:4].astype(int)

In [54]:
df_Bigtech = df_Bigtech[['season','year','amazon_lobby','facebook_lobby','google_lobby','apple_lobby','microsoft_lobby', \
                        'amazon_link','facebook_link','google_link','apple_link','microsoft_link']].reset_index(drop = True)

In [56]:
# df_Bigtech.info()

In [57]:
df_Bigtech

,season,year,amazon_lobby,facebook_lobby,google_lobby,apple_lobby,microsoft_lobby,amazon_link,facebook_link,google_link,apple_link,microsoft_link
0,2021 2nd Quarter - Report,2021,4860000.0,4770000.0,2090000.0,1640000.0,2470000.0,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...
1,2021 1st Quarter - Report,2021,4800000.0,4790000.0,2690000.0,1460000.0,2590000.0,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...
2,2020 4th Quarter - Report,2020,4740000.0,4690000.0,2110000.0,1450000.0,2190000.0,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...
3,2020 3rd Quarter - Report,2020,4410000.0,4900000.0,1930000.0,1560000.0,1880000.0,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...
4,2020 3rd Quarter - Amendment,2020,0.0,4900000.0,0.0,1560000.0,0.0,0,https://lda.senate.gov//filings/public/filing/...,0,https://lda.senate.gov//filings/public/filing/...,0
5,2020 2nd Quarter - Report,2020,4380000.0,4830000.0,1690000.0,1480000.0,2910000.0,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...
6,2020 1st Quarter - Report,2020,4330000.0,5260000.0,1800000.0,2160000.0,2394000.0,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...
7,2019 4th Quarter - Report,2019,4210000.0,4430000.0,2740000.0,1830000.0,2390000.0,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...
8,2019 4th Quarter - Amendment,2019,0.0,0.0,0.0,1890000.0,0.0,0,0,0,https://lda.senate.gov//filings/public/filing/...,0
9,2019 3rd Quarter - Report,2019,4030000.0,4770000.0,2770000.0,1780000.0,2250000.0,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...,https://lda.senate.gov//filings/public/filing/...


## ※ Check Amendment carefully. If needed, update numbers manually on .csv files. 

In [60]:
df_Bigtech.to_csv('bigtech_lobby.csv', index=False)

### (2) API version (only Amazon)

In [22]:
amas = [] 
for page in range(1,5):
    url = 'https://lda.senate.gov/api/v1/filings/?filing_uuid=&page={}&registrant_id=&registrant_name=Amazon'.format(page)
    res = requests.get(url)
    data = res.json()
    amas.append(data)

In [58]:
# amas[0]

In [24]:
am_url1 = 'https://lda.senate.gov/api/v1/filings/?filing_uuid=&page=1&registrant_id=&registrant_name=Amazon'
am_url2 = 'https://lda.senate.gov/api/v1/filings/?filing_uuid=&page=2&registrant_id=&registrant_name=Amazon'
am_url3 = 'https://lda.senate.gov/api/v1/filings/?filing_uuid=&page=3&registrant_id=&registrant_name=Amazon'
am_url4 = 'https://lda.senate.gov/api/v1/filings/?filing_uuid=&page=4&registrant_id=&registrant_name=Amazon'

In [25]:
res1 = requests.get(am_url1)
data1 = res1.json()
res2 = requests.get(am_url2)
data2 = res2.json()
res3 = requests.get(am_url3)
data3 = res3.json()
res4 = requests.get(am_url4)
data4 = res4.json()

In [26]:
# data.keys()
# print(data['count'])
# print(data['next'])
# data1['results'][1].keys()

In [27]:
amalobbies1 = data1['results']
amalobbies2 = data2['results']
amalobbies3 = data3['results']
amalobbies4 = data4['results']

In [28]:
amazons = []
for amalobby in amalobbies1:
    amazon = {}
    amazon['year'] = amalobby['filing_year'] 
    amazon['season'] = amalobby['filing_period']
    amazon['expenses'] = amalobby['expenses']
    amazon['url'] = amalobby['filing_document_url']
    amazons.append(amazon)
for amalobby in amalobbies2:
    amazon = {}
    amazon['year'] = amalobby['filing_year'] 
    amazon['season'] = amalobby['filing_period']
    amazon['expenses'] = amalobby['expenses']
    amazon['url'] = amalobby['filing_document_url']
    amazons.append(amazon)
for amalobby in amalobbies3:
    amazon = {}
    amazon['year'] = amalobby['filing_year'] 
    amazon['season'] = amalobby['filing_period']
    amazon['expenses'] = amalobby['expenses']
    amazon['url'] = amalobby['filing_document_url']
    amazons.append(amazon)
for amalobby in amalobbies4:
    amazon = {}
    amazon['year'] = amalobby['filing_year'] 
    amazon['season'] = amalobby['filing_period']
    amazon['expenses'] = amalobby['expenses']
    amazon['url'] = amalobby['filing_document_url']
    amazons.append(amazon)

In [1]:
# pd.DataFrame(amazons)